In [0]:
import requests
import pandas as pd

headers = {
    'accept': 'application/json',
    'x-access-token-ws': 'd1bf4238656c2167c5997e4081b549eedd0a25622a18740d8ad1a8a31ee22b15'
}

# -------------------- ANIMAL ---------------------------------------#
url = 'https://api.irancho.com.br/api/animal'
response = requests.get(url, headers=headers)
animals_list = response.json()
df_animal = pd.json_normalize(animals_list)
print(f'animal {response.status_code}')
